# Server Script

## Overview

### Database

The DynamoDB database should have the following columns:

- `id` (int): primary key created from the database
- `session_id` (string): hash created for each session in the front-end
- `answers` (serialized json): json from the app
- `resolved` (boolean): boolean storing whether the solution resolved the issue for that customer
- `jira_key` (string): key from the jira
- `zendesk_ticket_id` (int): id of the Zendesk ticket created for the troubleshooter
- `updated` (timestamp): time when the row was last updated

### Flow

Here is an overview of the data flow for the troubleshooter.

1. User arrives at troubleshooter. Unique `session_id` is created.
2. When user answers a question, a list of all their answers is sent to the lambda.
3. The lambda creates a row for the session, and stores the latest `answers` it has received
4. When the user reaches a terminal node in the tree, then the lambda stores the `jira_key` of the known issue in the database
5. If the user says that the solution provided did not resolve the problem, then the lambda takes the data and creates a Zendesk ticket, storing the resulting ticket id in `zendesk_ticket_id`

### JSON Examples

#### Case 1: Issue Not Identified

Initial JSON responses when the issue has **not yet been identified** will have `jira_key`, `resolved`, and `email` set to `null`.

```
{
    "session_id": "89d98eee66a278bb34d7bf548f23d7e9",
    "jira_key": null,
    "resolved": null,
    "email": null,
    "answers": [ { "question": "Is the sky blue?", "answer": "Yes" } ]
}
```

#### Case 2: Issue Identified + Not Sure If Resolved

When the issue has been identified, the `jira_key` will have a value. At this stage the user will be asked whether the workaround provided has solved the issue for them.

```
{
    "session_id": "89d98eee66a278bb34d7bf548f23d7e9",
    "jira_key": "OS-1234",
    "resolved": null,
    "email": null,
    "answers": [ { "question": "Is the sky blue?", "answer": "Yes" }, { "question": "What is the meaning of life?", "answer": "No" ]
}
```

#### Case 3: Issue Identified + Resolved

If the user responds that the workaround did resolve the issue, then the issue is marked as **resolved**. There is no email provided because we don't need it.

```
{
    "session_id": "89d98eee66a278bb34d7bf548f23d7e9",
    "jira_key": "OS-1234",
    "resolved": true,
    "email": null,
    "answers": [ { "question": "Is the sky blue?", "answer": "Yes" }, { "question": "What is the meaning of life?", "answer": "No" ]
}
```

#### Case 4: Issue Identified + Not Resolved --> Create ZenDesk Ticket

Final JSON response which will trigger a Zendesk ticket to be created

```
{
    "session_id": "89d98eee66a278bb34d7bf548f23d7e9",
    "jira_key": "OS-1234",
    "resolved": false,
    "email": "brandon@kano.me",
    "answers": [ { "question": "Is the sky blue?", "answer": "Yes" }, { "question": "What is the meaning of life?", "answer": "No" ]
}
```

## The Code

### The Handler Function

This function will take the data from the Amazon API Endpoint manager.


In [1]:
import json
import ast
from zenpy import Zenpy
from zenpy.lib.api_objects import Ticket
from zenpy.lib.api_objects import User
from zenpy.lib.api_objects import Comment

def handler(data):
    
    # write to the database
    updateDatabase(data)

    # create ticket
    if data["email"] is not None: 
        ticket_id = createTicket(data)
        updateDatabase(data, ticket_id)

### Update the Database

This will put the data from the JSON into the database, overwriting what was there before.

In [2]:
def updateDatabase(data, zendesk_ticket_id=None):
    
    row = {
        'session_id': data["session_id"],
        'answers': data["answers"],
        'resolved': data["resolved"],
        'jira_key': data["jira_key"],
        'zendesk_ticket_id': zendesk_ticket_id
    }
    
    
    # @todo implement code to write to database here
    # it should overwrite any existing data for the given session ID

### Create Zendesk Ticket

This will happen if the status is marked as unresolved and the user has given us their email address. It uses the ZenPy API, which is [documented here](http://docs.facetoe.com.au/zenpy.html#creating-updating-and-deleting-api-objects)

In [3]:
def createTicket(data):
    
    # Zenpy accepts an API token
    creds = {
        'email' : '',
        'token' : '',
        'subdomain': 'kano'
    }
    zenpy_client = Zenpy(**creds)
    
    # @TODO
    # Check the database to see if the row for this session already has a zendesk 
    # ticket ID for this session. If it does, don't create new zendesk ticket!
    # We don't want duplicates
    
    # Grab the last terminal from the answers list, which should contain diagnostics
    terminal = data["answers"][-1]
    if "diagnosis" not in terminal:
        raise BaseException("data['answers'] object does not contain a well-formed terminal node")
    
    
    # @TODO 
    # Associate with the right product in ZenDesk
    kit = getKitFromAnswers(data["answers"])
    
    # Create a Private Note Describing the Issue
    note = "This customer had an unresolved issue. They need your help!"
    note += "<h3>Summary</h3>"
    note += "<ul>"
    note += "<li><b>Diagnosis</b>: %s (Jira Issue <a href='https://kanocomputing.atlassian.net/browse/%s'>%s</a>)</li>" % (terminal["diagnosis"],terminal["jira_key"],terminal["jira_key"])
    note += "<li><b>Agent Solution</b>: %s</li>" % (terminal["agent_solution"])
    note += "<li><b>Solution Attempted By Customer (Unsuccessfully)</b>: %s</li>" % (terminal["customer_solution"])
    note += "</ul>"
    note += "<h3>Customer's Answers</h3>"
    note += "<ul>"
    for answer in data["answers"]:
        note += "<li>%s: %s</li>" % (answer["question"], answer["answer"])
    note += "</ul>"

    # Create Ticket in Zendesk
    ticket = Ticket(
        requester = User(email=data["email"]),
        subject = "Troubleshooter: " + terminal["diagnosis"],
        tags = ["troubleshooter"],
        comment = Comment(html_body=note, public=False)
    )

    audit = zenpy_client.tickets.create(ticket)
    
    # @TODO
    # Link with Jira Issue 
    # something like this: zenpy_client.jira_links.create()
    
    # @TODO
    # Specify the product

    return audit.ticket.id

## Helper Functions

In [4]:
def getKitFromAnswers(answers):
    kit = answers[0]["answer"]
    if kit=="Computer Kit Touch":
        return "Computer Kit Touch"
    elif kit=="Computer Kit Complete":
        return "Computer Kit Complete"
    elif kit=="Computer Kit":
        return "Computer Kit"
    elif kit=="Harry Potter Kano Coding Kit":
        return "Harry Potter Kano Coding Kit"
    elif kit=="Motion Sensor Kit":
        return "Motion Sensor Kit"
    elif kit=="Pixel Kit":
        return "Pixel Kit"
    else:
        raise BaseException("Kit not found in answers data. Check to make sure the first response contains a string with the kit name")

## Example: Create a ticket from Brandon

In [7]:
def example():
    json_string = '{"session_id":"akxsq68xk0v","email":"brandonscottjackson@gmail.com","answers":[{"question":"Which Kano kit are you having trouble with?","type":"question","options":{"Computer Kit Touch":1,"Harry Potter Kano Coding Kit":31,"Motion Sensor Kit":56},"answer":"Harry Potter Kano Coding Kit"},{"question":" Have you been able to open the Kano App successfully?","type":"question","options":{"Yes":32,"No":49},"answer":"Yes"},{"question":" Have you been able to connect your Wand to the App?","type":"question","options":{"Yes":33,"No":38},"answer":"Yes"},{"question":" When you open up a challenge on the Play tab, do you see a colorful world on the right side of your screen?","type":"question","options":{"Yes":34,"No":37},"answer":"Yes"},{"question":" Great! If you set the wand on a table, does the wand drift off the screen?","type":"question","options":{"Yes":35,"No":36},"answer":"Yes"},{"question":" Sorry to hear that. Let\'s get you a new wand.","type":"known_issue","diagnosis":" Wand drifting","customer_solution":" Put your wand on a table and leave it on the playground. It should stop drifting. If it doesn’t let us know!","jira_key":"HP-31","agent_solution":" Ask them to compare with baseline drift video. Replace if it does indeed drift more than it should.","answer":""}],"jira_key":"HP-31","resolved":null}'
    json_data = json.loads(json_string)
    handler(json_data)